In [1]:
from blackbox import BlackBox
import sys
import random
import math
import binascii

In [9]:
input_file_path = "/Users/gopi/Desktop/Assignment5/users.txt"
output_file_path = "/Users/gopi/Desktop/Assignment5/output_2.csv"
stream_size = 300
num_of_asks = 100
m = 997
bx = BlackBox()

In [10]:
def myhashs(user):
    user_id = int(binascii.hexlify(user.encode('utf8')),16)
    result = []
    a = 37
    b = 13
    for i in range(55):
        hash_value = ((a * user_id * i) + (b)) % m
        result.append(hash_value)
    return result

def flajolet_martin(users_stream):
    user_set = set()
    hash_values_all = []
    
    for user in users_stream:
        hv = myhashs(user)
        hash_values_all.append([bin(i)[2:] for i in hv])
        user_set.add(user)
    
    predictions = []
    for i in range(55):
        max_value = 0
        for j in range(len(users_stream)):
            num_of_zero = len(hash_values_all[j][i]) - len(hash_values_all[j][i].rstrip('0'))
            if num_of_zero > max_value:
                max_value = num_of_zero
        predictions.append(2**max_value)
    
    group_avgs = []
    #print(len(predictions))
    for i in range(11):
        group_avgs.append(sum(predictions[i*5 : (i*5)+5])/5)
    result=[0,0]
    result[0] = len(user_set)
    result[1] = int(group_avgs[6])
    return result
    

In [11]:
file = open(output_file_path, "w")
file.write("Time,Ground Truth,Estimation")
file.write("\n")
gt_sum = 0
pd_sum = 0
for i in range(num_of_asks):
    stream_users = bx.ask(input_file_path, stream_size)
    prediction = flajolet_martin(stream_users)
    gt_sum += prediction[0]
    pd_sum += prediction[1]
    string = str(i) + "," + str(prediction[0]) + "," + str(prediction[1]) + "\n"
    file.write(string)

file.close()
print(pd_sum/gt_sum)
    

0.8675578371891459
